In [4]:
from utils import *
import argparse
from snowflake_config import SNOW_CONFIG

def snsql(query, con):
    return pd.read_sql(query, con)

In [219]:
# Gets the version
ctx = snowflake.connector.connect(
    user = 'mliu-ext@beachbody.com',
    account = 'tbc-dev',
    authenticator = 'externalbrowser',
    role = 'DNA_DE',
    warehouse = 'ETL_WH'
    )
cs = ctx.cursor()
try:
    cs.execute("select current_warehouse(), current_database(), current_schema();")
    one_row = cs.fetchone()
    print(one_row)
finally:
    print()
    cs.close()
ctx.close()

('ETL_WH', None, None)



In [5]:
# create snowflake connector
con = sn_conn(SNOW_CONFIG)
con.cursor().execute("USE WAREHOUSE ETL_WH");

# print out the current role and wharehouse
query = """select current_role(), current_warehouse(), current_database(), current_schema()"""
print(snsql(query, con))

  CURRENT_ROLE() CURRENT_WAREHOUSE() CURRENT_DATABASE() CURRENT_SCHEMA()
0         DNA_DE              ETL_WH               None             None


In [5]:
full_tbl_path = 'SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.Call_Center'

table = TableStats(full_tbl_path, con)

In [6]:
table.col_count


31

In [7]:
def main():
    
    table = 'SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.Call_Center'
    
    prod = TableStats(table, con)
    dev = TableStats(table, con)
    
    TC = TableComp()
    res, df = TC.compare_all_cols(prod, dev)
    
    print("----------residual dataframe-------------")
    print(res)
    
    return res, df

In [8]:
# test
dfa = pd.DataFrame([['a', 1], ['b',2]], columns=['value', 'cnt2'])
dfb = pd.DataFrame([['a', 3], ['b',2]], columns=['value', 'cnt2'])
TC = TableComp()

res, df = TC.compare_group_count(dfa, dfb)

df

,value,cnt2,_merge
0,a,1,left_only
1,b,2,both
2,a,3,right_only


In [9]:
res, df = main()


Results will be saved in /Users/mliu-ext/downloads/2021-04-10_0244_SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.Call_Center_vs_SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.Call_Center.xlsx

Comparing two tables coloumn by column 



comparing column: CC_CALL_CENTER_SK
comparing column: CC_CALL_CENTER_ID
comparing column: CC_REC_START_DATE
comparing column: CC_REC_END_DATE
comparing column: CC_CLOSED_DATE_SK
comparing column: CC_OPEN_DATE_SK
comparing column: CC_NAME
comparing column: CC_CLASS
comparing column: CC_EMPLOYEES
comparing column: CC_SQ_FT
comparing column: CC_HOURS
comparing column: CC_MANAGER
comparing column: CC_MKT_ID
comparing column: CC_MKT_CLASS
comparing column: CC_MKT_DESC
comparing column: CC_MARKET_MANAGER
comparing column: CC_DIVISION
comparing column: CC_DIVISION_NAME
comparing column: CC_COMPANY
comparing column: CC_COMPANY_NAME
comparing column: CC_STREET_NUMBER
comparing column: CC_STREET_NAME
comparing column: CC_STREET_TYPE
comparing column: CC_SUITE_NUMBER
comparing column: CC_CITY
comparing column: CC_COUNTY
comparing column: CC_STATE
comparing column: CC_ZIP
comparing column: CC_COUNTRY
comparing column: CC_GMT_OFFSET
comparing column: CC_TAX_PERCENTAGE

----------residual dataframe-

In [6]:
table = 'SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.Call_Center_Test'

left = 'RAW.JING_VIPER.RAW_VIPER_20201110'
right = 'RAW.JING_VIPER.VIPER_20201110'

tbl_a = TableStats(left, con)
tbl_b = TableStats(right, con)

TC = TableComp()
df = TC.compare_count(tbl_a.row_count, tbl_b.row_count, 0)
df

,test_type,column_names,results,uniques_left,uniques_right,shared,diff_pct,threshold
0,numberOfRows,NaN,PASSED,2476,2476,NaN,0,0


In [7]:
TableStats(table, con)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



(<class 'pandas.io.sql.DatabaseError'>, DatabaseError("Execution failed on sql 'desc table SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.Call_Center_Test': 002003 (42S02): SQL compilation error:\nTable 'SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.CALL_CENTER_TEST' does not exist or not authorized."), <traceback object at 0x7f8b6c4841c0>)
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/py39/lib/python3.8/site-packages/pandas/io/sql.py", line 1681, in execute
    cur.execute(*args, **kwargs)
  File "/usr/local/anaconda3/envs/py39/lib/python3.8/site-packages/snowflake/connector/cursor.py", line 626, in execute
    Error.errorhandler_wrapper(self.connection, self,
  File "/usr/local/anaconda3/envs/py39/lib/python3.8/site-packages/snowflake/connector/errors.py", line 236, in errorhandler_wrapper
    cursor.errorhandler(connection, cursor, error_class, error_value)
  File "/usr/local/anaconda3/envs/py39/lib/python3.8/site-packages/snowflake/connector/errors.py", line 169, in default_err

TypeError: object of type 'NoneType' has no len()

In [48]:
try:
    pd.read_sql("desc table {}".format(table), con)
except:
#     raise SystemExit('error in code want to exit')
#     print(sys.exc_info())
#     sys.exit("something wrong")

    print("something wrong")
    exit()
#     sys.exit(0)
print('hello')

something wrong
hello


In [11]:
TC.compare_group_count(tbl_a.columns, tbl_b.columns)

(       test_type column_names results  uniques_left  uniques_right shared  \
 0  DistinctValue   [col_name]  PASSED            25             25     25   
 
   diff_pct threshold  
 0      NaN       NaN  ,
                   col_name _merge
 0                 TBC_UUID   both
 1            VPR_TIMESTAMP   both
 2           VPR_PROFILE_ID   both
 3           VPR_VIDEO_GUID   both
 4             VPR_TIMECODE   both
 5             VPR_DURATION   both
 6   VPR_HEARTBEAT_INTERVAL   both
 7             VPR_PLATFORM   both
 8     VPR_PLATFORM_DETAILS   both
 9           VPR_EVENT_TYPE   both
 10       VPR_EVENT_DETAILS   both
 11                 VPR_DST   both
 12                  VPR_IP   both
 13          VPR_SESSION_ID   both
 14     VPR_LOCAL_TIMESTAMP   both
 15          VPR_USER_AGENT   both
 16  VPR_TEALIUM_VISITOR_ID   both
 17  VPR_TEALIUM_SESSION_ID   both
 18        VPR_SUBTITLES_ON   both
 19         VPR_AUDIO_TRACK   both
 20      VPR_SUBTITLE_TRACK   both
 21      VPR_PLAYER_VIS

In [17]:
tbl_a.get_group_count("VPR_DURATION").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   vpr_duration  40 non-null     object
 1   cnt           40 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 768.0+ bytes


In [18]:
tbl_b.get_group_count("VPR_DURATION").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   vpr_duration  40 non-null     int64
 1   cnt           40 non-null     int64
dtypes: int64(2)
memory usage: 768.0 bytes


In [20]:
a = tbl_a.get_column_names_with_type().iloc[0,1]
import json
res = json.loads(a)
type(res)
res['type']
test = '{"type":"TEXT","length":16777216,"byteLength":16777216,"nullable":true,"fixed":false}'

json.loads(test)['type']

'TEXT'

In [22]:
tbl_b.get_column_names_with_type()

,col_name,type
0,TBC_UUID,"{""type"":""TEXT"",""length"":16777216,""byteLength"":..."
1,VPR_TIMESTAMP,"{""type"":""TEXT"",""length"":16777216,""byteLength"":..."
2,VPR_PROFILE_ID,"{""type"":""TEXT"",""length"":16777216,""byteLength"":..."
3,VPR_VIDEO_GUID,"{""type"":""TEXT"",""length"":16777216,""byteLength"":..."
4,VPR_TIMECODE,"{""type"":""TEXT"",""length"":16777216,""byteLength"":..."
5,VPR_DURATION,"{""type"":""FIXED"",""precision"":38,""scale"":0,""null..."
6,VPR_HEARTBEAT_INTERVAL,"{""type"":""FIXED"",""precision"":38,""scale"":0,""null..."
7,VPR_PLATFORM,"{""type"":""TEXT"",""length"":16777216,""byteLength"":..."
8,VPR_PLATFORM_DETAILS,"{""type"":""TEXT"",""length"":16777216,""byteLength"":..."
9,VPR_EVENT_TYPE,"{""type"":""TEXT"",""length"":16777216,""byteLength"":..."


In [25]:
if '': 
    print("h")